# Libraries

In [1]:
import json, requests, os, re, math, glob, aiohttp, asyncio, urllib3, warnings
import pandas as pd

warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=DeprecationWarning)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from tqdm.notebook import tqdm

# URL

In [2]:
base_url = 'https://www.tokopedia.com'
first_url = 'https://www.tokopedia.com/find/kulkas-2-pintu'

# Parameter

In [3]:
ROWS = 200 # jumlah maksimal data yang disediakan API Endpoint
PAGES = 0

# Buka Requests

In [4]:
S = requests.Session()

# Buka 1 Halaman

In [5]:
base_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
}

In [6]:
r = S.get(first_url, headers=base_headers)

In [7]:
r.text

'<!DOCTYPE html>\n<html lang="id" translate="no">\n  <head>\n    <title data-rh="true">Kulkas 2 Pintu Termurah - Promo &quot;DISKONHL&quot; 50rb!</title>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="initial-scale=1, minimum-scale=1, maximum-scale=5, user-scalable=yes, width=device-width">\n    <link rel="preconnect" href="https://assets.tokopedia.net" crossorigin><link rel="dns-prefetch" href="https://assets.tokopedia.net" nonce="1weGZHWO1U8YN6AhNxM62w==">\n<link rel="preconnect" href="https://gql.tokopedia.com" crossorigin><link rel="dns-prefetch" href="https://gql.tokopedia.com" nonce="1weGZHWO1U8YN6AhNxM62w==">\n<link rel="preconnect" href="https://googleads.g.doubleclick.net" crossorigin><link rel="dns-prefetch" href="https://googleads.g.doubleclick.net" nonce="1weGZHWO1U8YN6AhNxM62w==">\n<link rel="preconnect" href="https://www.google-analytics.com" crossorigin><link rel="dns-prefetch" href="https://www.google-analytics.com" nonce="1weGZHWO1U8YN6AhNxM62w==">\n<l

In [8]:
cookies = r.cookies

In [9]:
cookies

<RequestsCookieJar[Cookie(version=0, name='_UUID_NONLOGIN_', value='85961c77b985f3637eeb18259d14d160', port=None, port_specified=False, domain='.tokopedia.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=3187209600, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='_UUID_NONLOGIN_.sig', value='5Zu7qJI9sY91F9bnmR9jsTs248M', port=None, port_specified=False, domain='.tokopedia.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=3187209600, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='_abck', value='5EA0B2B53C7873C45978DD0778EBFB0D~-1~YAAQl6ssF/Fp3dmRAQAA8FdC6gx754LvooCVaBXjt8W8xnIAQNPw2867N/7awlUugOrFh0dj3zmf6ZMvXUXNtj9Bupndjoo4/4i7B9O1GeMnNdCYOdDF8PKclc4x7ecVdFwnYwsv305ZH9qsuxolDsjixkfre4F0CKsrcx6IuRkG/SjTleR0LGwfwMwWLwMz+NYCF011GZpM7sPaVnXAPn2TdwMvp83G7EKZ78PViXMiHii3nlr1jEr5aA2ML

# Dapatkan Parameter Dibutuhkan oleh API Endpoint

In [10]:
pattern = r'window\.__cache={(.*?)};'

In [11]:
matches = re.findall(pattern, r.text, re.DOTALL)

In [12]:
matches = '{' + matches[0] + '}'

In [13]:
m = json.loads(matches)

In [14]:
m['ROOT_QUERY']

{'isAuthenticatedV2': '',
 'RolloutFeatureVariants({"client_id":4,"iris_session_id":"","rev":0})': {'type': 'id',
  'generated': True,
  'id': '$ROOT_QUERY.RolloutFeatureVariants({"client_id":4,"iris_session_id":"","rev":0})',
  'typename': 'RolloutFeatureVariantsResponse'},
 'trending_keywords': {'type': 'id',
  'generated': True,
  'id': '$ROOT_QUERY.trending_keywords',
  'typename': 'TrendingKeywords'},
 'findInfo({"params":"alias=kulkas-2-pintu"})': {'type': 'id',
  'generated': True,
  'id': '$ROOT_QUERY.findInfo({"params":"alias=kulkas-2-pintu"})',
  'typename': 'FindInfoData'},
 'RolloutFeatureVariants({"client_id":4,"id":"85961c77b985f3637eeb18259d14d160","iris_session_id":"","rev":0})': {'type': 'id',
  'generated': True,
  'id': '$ROOT_QUERY.RolloutFeatureVariants({"client_id":4,"id":"85961c77b985f3637eeb18259d14d160","iris_session_id":"","rev":0})',
  'typename': 'RolloutFeatureVariantsResponse'},
 'filter_sort_product({"params":"q=kulkas+2+pintu&source=search_product"})': {

In [15]:
rollout_ = ''
iris_ = ''
params_ = ''
adparams_ = ''
for key, value in m['ROOT_QUERY'].items():
    if 'RolloutFeature' in key:
        rollout_ = key
        iris_ = rollout_.split('iris_session_id":"')[1].split('","')[0]

    if'searchProductV5' in key:
        params_ = key.replace('searchProductV5({"params":"', '').replace('"})', '')

    if 'displayAdsV3' in key:
        adparams_ = key.replace('displayAdsV3({"displayParams":"', '').replace('"})', '')

In [16]:
iris_

''

In [17]:
rollout_

'RolloutFeatureVariants({"client_id":4,"id":"85961c77b985f3637eeb18259d14d160","iris_session_id":"","rev":0})'

In [18]:
params_

'device=desktop&fcity=&l_name=sre&ob=23&page=1&q=kulkas%202%20pintu&related=true&rows=60&safe_search=false&scheme=https&source=search&st=product&start=0&unique_id=85961c77b985f3637eeb18259d14d160&user_addressId=&user_cityId=176&user_districtId=2274&user_lat=&user_long=&user_postCode='

In [19]:
adparams_

'device=desktop&ep=product&fcity=&item=20&l_name=sre&minimum_item=10&ob=23&page=1&q=kulkas%202%20pintu&src=find&st=product'

# Data untuk dikirimkan ke API Endpoint

In [20]:
data_pre = [{"operationName":"SearchProductQueryV5","query":"query SearchProductQueryV5($params: String!, $adParams: String!) {\n  organic: searchProductV5(params: $params) {\n    header {\n      totalData\n      responseCode\n      keywordProcess\n      keywordIntention\n      componentID\n      isQuerySafe\n      additionalParams\n      backendFilters\n      __typename\n    }\n    data {\n      totalDataText\n      related {\n        relatedKeyword\n        position\n        trackingOption\n        otherRelated {\n          keyword\n          url\n          componentID\n          products {\n            id\n            name\n            url\n            applink\n            mediaURL {\n              image\n              __typename\n            }\n            shop {\n              id\n              name\n              city\n              tier\n              __typename\n            }\n            badge {\n              id\n              title\n              url\n              __typename\n            }\n            price {\n              text\n              number\n              __typename\n            }\n            freeShipping {\n              url\n              __typename\n            }\n            labelGroups {\n              position\n              title\n              type\n              url\n              styles {\n                key\n                value\n                __typename\n              }\n              __typename\n            }\n            rating\n            wishlist\n            ads {\n              id\n              productClickURL\n              productViewURL\n              productWishlistURL\n              tag\n              __typename\n            }\n            meta {\n              warehouseID\n              componentID\n              __typename\n            }\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      suggestion {\n        currentKeyword\n        suggestion\n        query\n        text\n        componentID\n        trackingOption\n        __typename\n      }\n      products {\n        id\n        name\n        url\n        applink\n        mediaURL {\n          image\n          image300\n          videoCustom\n          __typename\n        }\n        shop {\n          id\n          name\n          url\n          city\n          tier\n          __typename\n        }\n        badge {\n          id\n          title\n          url\n          __typename\n        }\n        price {\n          text\n          number\n          range\n          original\n          discountPercentage\n          __typename\n        }\n        freeShipping {\n          url\n          __typename\n        }\n        labelGroups {\n          position\n          title\n          type\n          url\n          styles {\n            key\n            value\n            __typename\n          }\n          __typename\n        }\n        category {\n          id\n          name\n          breadcrumb\n          gaKey\n          __typename\n        }\n        rating\n        wishlist\n        ads {\n          id\n          productClickURL\n          productViewURL\n          productWishlistURL\n          tag\n          __typename\n        }\n        meta {\n          countReview\n          parentID\n          warehouseID\n          isImageBlurred\n          isPortrait\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  topads: displayAdsV3(displayParams: $adParams) {\n    data {\n      id\n      productClickURL: product_click_url\n      product {\n        id\n        name\n        wishlist\n        image {\n          imageURL: s_ecs\n          viewTrackURL: s_url\n          __typename\n        }\n        url: uri\n        price: price_format\n        rating: product_rating\n        ratingAverage: product_rating_format\n        countReview: count_review_format\n        labelGroups: label_group {\n          position\n          title\n          type\n          url\n          style {\n            key\n            value\n            __typename\n          }\n          __typename\n        }\n        campaign {\n          discountPercentage: discount_percentage\n          originalPrice: original_price\n          __typename\n        }\n        category_breadcrumb\n        __typename\n      }\n      shop {\n        id\n        name\n        uri\n        location\n        goldmerchant: gold_shop\n        official: shop_is_official\n        badges {\n          title\n          imageURL: image_url\n          show\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"}]

In [21]:
data_pre

[{'operationName': 'SearchProductQueryV5',
  'query': 'query SearchProductQueryV5($params: String!, $adParams: String!) {\n  organic: searchProductV5(params: $params) {\n    header {\n      totalData\n      responseCode\n      keywordProcess\n      keywordIntention\n      componentID\n      isQuerySafe\n      additionalParams\n      backendFilters\n      __typename\n    }\n    data {\n      totalDataText\n      related {\n        relatedKeyword\n        position\n        trackingOption\n        otherRelated {\n          keyword\n          url\n          componentID\n          products {\n            id\n            name\n            url\n            applink\n            mediaURL {\n              image\n              __typename\n            }\n            shop {\n              id\n              name\n              city\n              tier\n              __typename\n            }\n            badge {\n              id\n              title\n              url\n              __typename\n   

In [22]:
data = data_pre.copy()
data[0]['variables'] = {"params":params_,"adParams":adparams_}

In [23]:
data

[{'operationName': 'SearchProductQueryV5',
  'query': 'query SearchProductQueryV5($params: String!, $adParams: String!) {\n  organic: searchProductV5(params: $params) {\n    header {\n      totalData\n      responseCode\n      keywordProcess\n      keywordIntention\n      componentID\n      isQuerySafe\n      additionalParams\n      backendFilters\n      __typename\n    }\n    data {\n      totalDataText\n      related {\n        relatedKeyword\n        position\n        trackingOption\n        otherRelated {\n          keyword\n          url\n          componentID\n          products {\n            id\n            name\n            url\n            applink\n            mediaURL {\n              image\n              __typename\n            }\n            shop {\n              id\n              name\n              city\n              tier\n              __typename\n            }\n            badge {\n              id\n              title\n              url\n              __typename\n   

# Panggil API Endpoint sekali, untuk mendapatkan total data

In [24]:
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
    'content-type': 'application/json',
    'Tkpd-UserId': '0',
    'x-device': 'desktop-0.0',
    'x-dark-mode': 'false',
    'X-Source': 'tokopedia-lite',
    'X-Version': 'ed69c5d',
    'X-Tkpd-Lite-Service': 'zeus',
    'iris-session_id': iris_,
    'Priority': 'u=4',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

In [25]:
r = S.post('https://gql.tokopedia.com/graphql/SearchProductQueryV5', headers=headers, json=data)

In [26]:
total_data = r.json()[0]['data']['organic']['header']['totalData']

In [27]:
products = r.json()[0]['data']['organic']['data']['products']

In [28]:
len(products)

60

In [29]:
total_data

441471

In [30]:
PAGES = math.ceil(total_data // ROWS)

In [31]:
PAGES

2207

In [32]:
#r.json()[0]['data']['organic']

# Try to get page 31

maximum data provided = 30 * 200 = 6000

In [32]:
page = 31
rows = ROWS
start = (page-1) * rows
data = data_pre.copy()
data[0]['variables'] = {
    "params":params_.\
        replace('page=1', 'page=' + str(page)).\
        replace('start=0', 'start=' + str(start)).\
        replace('rows=60', 'rows=' + str(rows)),
    "adParams":adparams_.replace('page=1', 'page=' + str(page))
}

In [33]:
data

[{'operationName': 'SearchProductQueryV5',
  'query': 'query SearchProductQueryV5($params: String!, $adParams: String!) {\n  organic: searchProductV5(params: $params) {\n    header {\n      totalData\n      responseCode\n      keywordProcess\n      keywordIntention\n      componentID\n      isQuerySafe\n      additionalParams\n      backendFilters\n      __typename\n    }\n    data {\n      totalDataText\n      related {\n        relatedKeyword\n        position\n        trackingOption\n        otherRelated {\n          keyword\n          url\n          componentID\n          products {\n            id\n            name\n            url\n            applink\n            mediaURL {\n              image\n              __typename\n            }\n            shop {\n              id\n              name\n              city\n              tier\n              __typename\n            }\n            badge {\n              id\n              title\n              url\n              __typename\n   

In [34]:
r = S.post('https://gql.tokopedia.com/graphql/SearchProductQueryV5', headers=headers, json=data)

In [35]:
data_scraped = r.json()[0]['data']['organic']['data']['products']

In [36]:
len(data_scraped)

0

In [38]:
if len(data_scraped) > 0:
    data_scraped[90]

# Convert to Pandas

In [139]:
li = []

for datum in data_scraped:
    li.append({
        'id': datum['id'],
        'name': datum['name'],
        'price': datum['price']['number'],
        'url': datum['url'],
        'shop_id': datum['shop']['id'],
        'shop_name': datum['shop']['name'],
        'shop_city': datum['shop']['city'],
        'shop_url': datum['shop']['url'],
        'category_id': datum['category']['id'],
        'category_name': datum['category']['name'],
        'category_url': base_url + '/p/' + datum['category']['breadcrumb'],
    })
df = pd.DataFrame(li)

In [140]:
df

,id,name,price,url,shop_id,shop_name,shop_city,shop_url,category_id,category_name,category_url
0,797919929,LG GN-B215SQMT Kulkas 2 Pintu Smart Inverter -...,3895000,https://www.tokopedia.com/newmega/lg-gn-b215sq...,130623,New Mega,Jakarta Timur,https://www.tokopedia.com/newmega,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
1,1054422604,"Kulkas 2 Pintu Samsung RT25 Digital Inverter, ...",5562000,https://www.tokopedia.com/dutacikarang/kulkas-...,2045456,duta elektronik cikarang,Kab. Bekasi,https://www.tokopedia.com/dutacikarang,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
2,13151062064,LG GNB222SQ IB KULKAS LG 2 PINTU INVERTER GRAT...,3729000,https://www.tokopedia.com/topsellofficial/lg-g...,8385223,TOPSELL OFFICIAL,Surabaya,https://www.tokopedia.com/topsellofficial,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
3,3487140381,LG GN G222SLCB Kulkas 2 Pintu Smart Inverter,4449000,https://www.tokopedia.com/e3elektronik/lg-gn-g...,1058931,E3 Shop Elektronik,Jakarta Utara,https://www.tokopedia.com/e3elektronik,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
4,13829175130,SAMSUNG RT42CG6420B1 KULKAS UNGKEP 2 PINTU 415...,8399000,https://www.tokopedia.com/bangunelektronik/sam...,1739006,BANGUN ELEKTRONIK,Jakarta Barat,https://www.tokopedia.com/bangunelektronik,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
...,...,...,...,...,...,...,...,...,...,...,...
195,1833369483,KULKAS SHARP SJ 237 MG 2 PINTU,2935000,https://www.tokopedia.com/grosirelektron/kulka...,2712217,Grosir Elektronik123,Bandung,https://www.tokopedia.com/grosirelektron,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
196,9813347394,Samsung Kulkas Side By Side 2 Pintu 647 Liter ...,12849000,https://www.tokopedia.com/electroniccityindone...,10389203,Electronic City Indonesia,Jakarta Selatan,https://www.tokopedia.com/electroniccityindonesia,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
197,7804930502,KULKAS SHARP SJ-450GP SD 2 PINTU INVERTER PLAS...,4850000,https://www.tokopedia.com/setia-mandiri/kulkas...,3276732,Setia Mandiri Elektronik,Jakarta Barat,https://www.tokopedia.com/setia-mandiri,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
198,940396009,ELECTROLUX ETB5400B-A Kulkas Inverter 2 Pintu ...,11299000,https://www.tokopedia.com/indahhomeshop/electr...,3646536,Indah Home Shop,Jakarta Selatan,https://www.tokopedia.com/indahhomeshop,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...


# Create a Directory

In [33]:
dir_path = 'scrape_tokopedia_requests'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [34]:
files = glob.glob(os.path.join(dir_path, '*'))
for file in files:
    try:
        os.remove(file)
    except: pass

# Scrape Page 1 to 30, 200 data from each page

In [35]:
rows = ROWS
start_page = 1
last_page = 30

pbar = tqdm(total=last_page-start_page+1)

for page in range(start_page, last_page+1):
    start = (page-1) * rows
    data = data_pre.copy()
    data[0]['variables'] = {
        "params":params_.\
            replace('page=1', 'page=' + str(page)).\
            replace('start=0', 'start=' + str(start)).\
            replace('rows=60', 'rows=' + str(rows)),
        "adParams":adparams_.replace('page=1', 'page=' + str(page))
    }
    
    r = S.post('https://gql.tokopedia.com/graphql/SearchProductQueryV5', headers=headers, json=data)

    filename = f'{dir_path}/page_{page}.json'
    with open(filename, 'w') as f:
        json.dump(r.json(), f, indent=4)

    pbar.update(1)

pbar.close()

  0%|          | 0/30 [00:00<?, ?it/s]

# Combine ALL JSONs

In [36]:
all_files = os.listdir(dir_path)

json_files = [file for file in all_files if file.endswith('.json')]

In [37]:
li = []
pbar = tqdm(total=len(json_files))

for json_file in json_files:
    file_path = os.path.join(dir_path, json_file)
    with open(file_path) as f:
        data = json.load(f)[0]['data']['organic']['data']['products']

        for datum in data:
            li.append({
            'id': datum['id'],
            'name': datum['name'],
            'price': datum['price']['number'],
            'url': datum['url'],
            'shop_id': datum['shop']['id'],
            'shop_name': datum['shop']['name'],
            'shop_city': datum['shop']['city'],
            'shop_url': datum['shop']['url'],
            'category_id': datum['category']['id'],
            'category_name': datum['category']['name'],
            'category_url': base_url + '/p/' + datum['category']['breadcrumb'],
            })

    pbar.update(1)

df = pd.DataFrame(li)

pbar.close()

df.to_excel(os.path.join(dir_path, 'hasil_scrape.xlsx'), index=False)

  0%|          | 0/30 [00:00<?, ?it/s]

In [38]:
df

,id,name,price,url,shop_id,shop_name,shop_city,shop_url,category_id,category_name,category_url
0,10897107002,AQUA Elektronik Kulkas 2 Pintu 169 L AQR-D251,2599000,https://www.tokopedia.com/aquaelektronik/aqua-...,8061075,AQUA Elektronik Indonesia,Jakarta Utara,https://www.tokopedia.com/aquaelektronik,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
1,10681292159,LG GN-B222SQIB Kulkas 2 Pintu 225L Smart Inver...,3589000,https://www.tokopedia.com/lg-authorized-dealer...,3746415,LG authorized dealer,Jakarta Utara,https://www.tokopedia.com/lg-authorized-dealer,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
2,7831740717,CHANGHONG KULKAS 2 PINTU FTM155DB [122L],1999000,https://www.tokopedia.com/changhong-1/changhon...,7223586,Changhong Home Appliance,Kab. Tangerang,https://www.tokopedia.com/changhong-1,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
3,10681203961,LG GN-B202SQIB Kulkas 2 Pintu 202L Smart Inver...,3399000,https://www.tokopedia.com/lg-authorized-dealer...,3746415,LG authorized dealer,Jakarta Utara,https://www.tokopedia.com/lg-authorized-dealer,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
4,9586636083,Samsung Kulkas 2 Pintu 203 L RT19M dengan All-...,3899000,https://www.tokopedia.com/samsungelectronicsst...,11694144,Samsung Electronics Authorized Store,Jakarta Barat,https://www.tokopedia.com/samsungelectronicsst...,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
...,...,...,...,...,...,...,...,...,...,...,...
5995,12259633776,Modena Kulkas 2 pintu RF 2250 TGDS RF2250TGDS ...,3315000,https://www.tokopedia.com/florenelectronic/mod...,932846,Floren Electronic,Jakarta Selatan,https://www.tokopedia.com/florenelectronic,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
5996,623977624,AQUA SANYO KULKAS 2 PINTU AQRD270 AQRD 270 AQR...,2945000,https://www.tokopedia.com/sgutama/aqua-sanyo-k...,7191338,SGUTAMA,Tangerang,https://www.tokopedia.com/sgutama,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
5997,10493280419,Kulkas Inverter 2 Pintu LG GN-B222SQIB 225L LG...,3690000,https://www.tokopedia.com/bogorsolusi/kulkas-i...,7777056,Bogor Solusi Elektronik,Bogor,https://www.tokopedia.com/bogorsolusi,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
5998,6191357059,Relay Coil 1/4 pk Kulkas 2 pintu PTC Gulungan ...,15000,https://www.tokopedia.com/citramakmurshop/rela...,3902893,Citra Makmur Olshop,Tangerang,https://www.tokopedia.com/citramakmurshop,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...


# Async

## Create a Directory

In [32]:
dir_path = 'scrape_tokopedia_requests_aiohttp'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [33]:
files = glob.glob(os.path.join(dir_path, '*'))
for file in files:
    try:
        os.remove(file)
    except: pass

## Scrape Page 1 to 30, 200 data from each page, total data = 6000

In [34]:
all_data = []

rows = ROWS
start_page = 1
last_page = 2

for page in range(start_page, last_page+1):
    start = (page-1) * rows
    data = data_pre.copy()
    data[0]['variables'] = {
        "params":params_.\
            replace('page=1', 'page=' + str(page)).\
            replace('start=0', 'start=' + str(start)).\
            replace('rows=60', 'rows=' + str(rows)),
        "adParams":adparams_.replace('page=1', 'page=' + str(page))
    }

    all_data.append(data)


In [35]:
all_data

[[{'operationName': 'SearchProductQueryV5',
   'query': 'query SearchProductQueryV5($params: String!, $adParams: String!) {\n  organic: searchProductV5(params: $params) {\n    header {\n      totalData\n      responseCode\n      keywordProcess\n      keywordIntention\n      componentID\n      isQuerySafe\n      additionalParams\n      backendFilters\n      __typename\n    }\n    data {\n      totalDataText\n      related {\n        relatedKeyword\n        position\n        trackingOption\n        otherRelated {\n          keyword\n          url\n          componentID\n          products {\n            id\n            name\n            url\n            applink\n            mediaURL {\n              image\n              __typename\n            }\n            shop {\n              id\n              name\n              city\n              tier\n              __typename\n            }\n            badge {\n              id\n              title\n              url\n              __typename\n 

In [36]:
async def get_json(session, url, data, headers):
    async with session.post(url, headers=headers, json=data) as response:
        try:
            return await response.json(content_type=None)
        except: return {}

In [37]:
async def main():
    global cookies
    global headers
    global all_data

    timeout = aiohttp.ClientTimeout(total=None)

    async with aiohttp.ClientSession(cookies=cookies, timeout=timeout, connector=aiohttp.TCPConnector()) as session:
        tasks = []
        for data in all_data:
            tasks.append(asyncio.ensure_future(get_json(session, 'https://gql.tokopedia.com/graphql/SearchProductQueryV5', data, headers)))

        responses = await asyncio.gather(*tasks)
        for response in responses:
            filename = f'{dir_path}/page_{response[0]["data"]["organic"]["data"]["products"]["id"]}.json'
            #filename = f'{dir_path}/page_{response[0]["data"]["organic"]["header"]["page"]}.json'
            #[0]['data']['organic']['data']['products']['id']
            with open(filename, 'w') as f:
                json.dump(response, f, indent=4)
        #return await asyncio.gather(*tasks)
    

In [38]:
await main()

In [ ]:
rows = ROWS
start_page = 1
last_page = 2

pbar = tqdm(total=last_page-start_page+1)

for page in range(start_page, last_page+1):
    start = (page-1) * rows
    data = data_pre.copy()
    data[0]['variables'] = {
        "params":params_.\
            replace('page=1', 'page=' + str(page)).\
            replace('start=0', 'start=' + str(start)).\
            replace('rows=60', 'rows=' + str(rows)),
        "adParams":adparams_.replace('page=1', 'page=' + str(page))
    }
    
    r = S.post('https://gql.tokopedia.com/graphql/SearchProductQueryV5', headers=headers, json=data)

    filename = f'{dir_path}/page_{page}.json'
    with open(filename, 'w') as f:
        json.dump(r.json(), f, indent=4)

    pbar.update(1)

pbar.close()

  0%|          | 0/2 [00:00<?, ?it/s]

# Combine ALL JSONs

In [ ]:
all_files = os.listdir(dir_path)

json_files = [file for file in all_files if file.endswith('.json')]

In [ ]:
li = []
pbar = tqdm(total=len(json_files))

for json_file in json_files:
    file_path = os.path.join(dir_path, json_file)
    with open(file_path) as f:
        data = json.load(f)[0]['data']['organic']['data']['products']

        for datum in data:
            li.append({
            'id': datum['id'],
            'name': datum['name'],
            'price': datum['price']['number'],
            'url': datum['url'],
            'shop_id': datum['shop']['id'],
            'shop_name': datum['shop']['name'],
            'shop_city': datum['shop']['city'],
            'shop_url': datum['shop']['url'],
            'category_id': datum['category']['id'],
            'category_name': datum['category']['name'],
            'category_url': base_url + '/p/' + datum['category']['breadcrumb'],
            })

    pbar.update(1)

df = pd.DataFrame(li)

pbar.close()

df.to_excel(os.path.join(dir_path, 'hasil_scrape.xlsx'), index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df

,id,name,price,url,shop_id,shop_name,shop_city,shop_url,category_id,category_name,category_url
0,7831740717,CHANGHONG KULKAS 2 PINTU FTM155DB [122L],1999000,https://www.tokopedia.com/changhong-1/changhon...,7223586,Changhong Home Appliance,Kab. Tangerang,https://www.tokopedia.com/changhong-1,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
1,10897107002,AQUA Elektronik Kulkas 2 Pintu 169 L AQR-D251,2599000,https://www.tokopedia.com/aquaelektronik/aqua-...,8061075,AQUA Elektronik Indonesia,Jakarta Utara,https://www.tokopedia.com/aquaelektronik,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
2,10681292159,LG GN-B222SQIB Kulkas 2 Pintu 225L Smart Inver...,3589000,https://www.tokopedia.com/lg-authorized-dealer...,3746415,LG authorized dealer,Jakarta Utara,https://www.tokopedia.com/lg-authorized-dealer,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
3,10681203961,LG GN-B202SQIB Kulkas 2 Pintu 202L Smart Inver...,3399000,https://www.tokopedia.com/lg-authorized-dealer...,3746415,LG authorized dealer,Jakarta Utara,https://www.tokopedia.com/lg-authorized-dealer,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
4,9586636083,Samsung Kulkas 2 Pintu 203 L RT19M dengan All-...,3899000,https://www.tokopedia.com/samsungelectronicsst...,11694144,Samsung Electronics Authorized Store,Jakarta Barat,https://www.tokopedia.com/samsungelectronicsst...,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
...,...,...,...,...,...,...,...,...,...,...,...
395,2024405720,Sharp Kulkas 2 Pintu SJ-326GC-SD/SR / SJ 326 G...,3899000,https://www.tokopedia.com/bonbon-elektronik/sh...,10569663,Bonbon_Elektronik,Jakarta Utara,https://www.tokopedia.com/bonbon-elektronik,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
396,15041481885,KULKAS SHARP SJ 316 MG LEMARI ES 2 PINTU SJ316...,3198000,https://www.tokopedia.com/mjgrosirelek/kulkas-...,5188958,MJ GROSIR ELEKTRONIK,Bekasi,https://www.tokopedia.com/mjgrosirelek,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
397,690452947,KULKAS GN-C272SXCN 2 PINTU LG INVERTER GNC272S...,5399900,https://www.tokopedia.com/apelectric/kulkas-gn...,533466,AP Electric,Tangerang Selatan,https://www.tokopedia.com/apelectric,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...
398,1297678539,Sharp Kulkas 2 pintu SJ-456GI-MK New J-Tech In...,6200000,https://www.tokopedia.com/sharpostangerang/sha...,3744660,Sharp Official Store Tangerang,Kab. Tangerang,https://www.tokopedia.com/sharpostangerang,60,Elektronik,https://www.tokopedia.com/p/elektronik/elektro...


# End